In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import torch
from torch import nn

In [5]:
from src.networks.core.tensor_shape import TensorShape
from src.networks.skip_nets.modulated_additive_skip_net import ModulatedAdditiveSkipNet
from src.networks.core.net import Net
from src.networks.skip_nets.modulated_dense_skip_net import ModulatedDenseSkipNet
from src.networks.core.parallel_net import ParallelNet
from src.module_analysis import count_parameters
from src.networks.skip_nets.dense_skip_net import DenseSkipNet
from src.networks.core.layer_connections import LayerConnections
from src.networks.skip_nets.additive_skip_net import AdditiveSkipNet

def create_mod_dense(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, 2))
    else:
        return Net.as_net(nn.Identity())

def create_mod_additive(from_idx: int, to_idx: int, num_f: int):
    if from_idx != to_idx:
        return Net.as_net(create_ones_linear(num_f, num_f))
    else:
        return Net.as_net(nn.Identity())

def create_ones_linear(in_features, out_features):
    l = nn.Linear(in_features, out_features)
    l.weight.data[:] = 1.0
    l.bias.data[:] = 0.0
    return l

def create_layer(i, is_last, in_features, out_features):
    return nn.Sequential(
        create_ones_linear(in_features, out_features),
        nn.CELU() if not is_last else nn.Identity(),
    )

num_features=3
# net = AdditiveSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=4,
#     num_features=num_features,
#     layer_connections='full',
#     initial_direct_connection_weight=1.0,
#     initial_skip_connection_weight=0.01,
#     return_dense=False,
#     weights_trainable=False,
# )
# net = DenseSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     num_layers=10,
#     num_features=num_features,
#     layer_connections='full'
# )
# net = ParallelNet.from_layer_provider(create_layer, in_size=num_features, out_sizes=[5, 7, 11, 13])
net = ModulatedDenseSkipNet.from_layer_provider(
    layer_provider=create_layer,
    modulator_provider=create_mod_dense,
    in_size=num_features,
    layer_out_sizes=[11, 23, 17, 13],
)
# net = ModulatedAdditiveSkipNet.from_layer_provider(
#     layer_provider=create_layer,
#     modulator_provider=create_mod_additive,
#     num_features=num_features,
#     num_layers=4,
#     # return_dense=True,
# )
print(net.count_parameters())
print(net)

out = net(torch.ones(111, 5, 19, num_features))
print(out.shape)
# print(out)
# print(net.forward_shape(TensorShape(features=num_features, s_0=111, s_1=5)))

1390
ModulatedDenseSkipNet(
  (layers): NetList(
    (0): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=3, out_features=11, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (1): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=13, out_features=23, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (2): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=27, out_features=17, bias=True)
        (1): CELU(alpha=1.0)
      )
    )
    (3): TorchNet(
      (torch_module): Sequential(
        (0): Linear(in_features=23, out_features=13, bias=True)
        (1): Identity()
      )
    )
  )
  (connection_modulators[0][0]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[1][0]): TorchNet(
    (torch_module): Linear(in_features=3, out_features=2, bias=True)
  )
  (connection_modulators[1][1]): TorchNet(
    (torch_module): Identity()
  )
  (connection_modulators[2][0]): TorchNet

In [6]:
90 * 3 + 3 * 3 * 18 + 3 * 3 * 3 + 3 * 3 * 1

468

In [7]:
3 * (90 + 18 + 3 + 1) + 468

804

In [8]:
804

804

In [9]:
from src.networks.core.torch_wrappers.conv_net import Conv2dNet

conv = Conv2dNet(2, 3, 4, padding='same')
conv

Conv2dNet(2, 3, kernel_size=(4, 4), stride=(1, 1), padding=same)

In [10]:
from src.networks.core.torch_wrappers.linear_net import LinearNet

lin = LinearNet(2, 3)
lin(torch.ones(2))

tensor([ 0.4794, -0.7732, -0.1890], grad_fn=<ViewBackward0>)

In [11]:
from src.networks.multihead_self_attention import MultiheadSelfAttention

mhsa = MultiheadSelfAttention(10, 5)
mhsa.forward_shape(TensorShape(features=10, s_0=None))
mhsa(torch.ones(123, 9, 10))

tensor([[[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         ...,
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041]],

        [[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         ...,
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041]],

        [[ 0.1577, -0.4909, -0.1444,  ...,  0.1137, -0.1642,  0.0041],
         [ 0.1577, -0.4909, -0.1444,  ...,  0

In [12]:
from src.networks.core.layer_connections import *
from src.integer_sequences import *

LayerConnectionsBuilder(12).sequential().every_step_receives(factorial).every_step_to_output().to_np()

array([[ 0,  0],
       [ 0, 12],
       [ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 1,  5],
       [ 1,  6],
       [ 1,  7],
       [ 1,  8],
       [ 1,  9],
       [ 1, 10],
       [ 1, 11],
       [ 1, 12],
       [ 2,  2],
       [ 2,  3],
       [ 2,  4],
       [ 2,  5],
       [ 2,  6],
       [ 2,  7],
       [ 2,  8],
       [ 2,  9],
       [ 2, 10],
       [ 2, 11],
       [ 2, 12],
       [ 3,  3],
       [ 3, 12],
       [ 4,  4],
       [ 4, 12],
       [ 5,  5],
       [ 5, 12],
       [ 6,  6],
       [ 6,  7],
       [ 6,  8],
       [ 6,  9],
       [ 6, 10],
       [ 6, 11],
       [ 6, 12],
       [ 7,  7],
       [ 7, 12],
       [ 8,  8],
       [ 8, 12],
       [ 9,  9],
       [ 9, 12],
       [10, 10],
       [10, 12],
       [11, 11],
       [11, 12],
       [12, 12]], dtype=int32)

In [10]:
%load_ext autoreload
%autoreload 2

from src.reinforcement_learning.reinforce.reinforce import Reinforce
import torch
from torch.distributions import Categorical
from torch import optim, nn
from src.reinforcement_learning.a2c.a2c import A2C
from src.networks.core.seq_net import SeqNet
import gymnasium as gym

env = gym.make("CartPole-v1")

def init_network(out_size: int):
    return SeqNet.from_layer_provider(
        layer_provider=lambda layer_nr, is_last_layer, in_features, out_features: nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.CELU()
        ),
        in_size=4,
        out_sizes=[32, 32, 32, out_size]
    )

def select_action(logits: torch.Tensor):
    dist = Categorical(logits=logits)
    
    action = dist.sample()
    
    return int(action), dist.log_prob(action) 

actor = init_network(2)
critic = init_network(1)

A2C(
    env=env,
    actor_network=actor,
    actor_network_optimizer=optim.Adam(actor.parameters()),
    critic_network=critic,
    critic_network_optimizer=optim.Adam(critic.parameters()),
    critic_loss=nn.MSELoss(),
    select_action=select_action,
    on_episode_done=lambda a2c, i_episode, is_best_episode, reward: print(f'{i_episode:>4}: {reward:>5.1f}'),
).find_optimal_policy(1000)

# Reinforce(
#     env=env,
#     policy_network=actor,
#     policy_network_optimizer=optim.Adam(actor.parameters()),
#     select_action=select_action,
#     on_episode_done=lambda a2c, is_best_episode, reward: print(reward)
# ).find_optimal_policy(1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
   0:  21.0
   1:  30.0
   2:  12.0
   3:  43.0
   4:  13.0
   5:   8.0
   6:  15.0
   7:  23.0
   8:  32.0
   9:  18.0
  10:   8.0
  11:  20.0
  12:  16.0
  13:  20.0
  14:  14.0
  15:  23.0
  16:  30.0
  17:  40.0
  18:  16.0
  19:  10.0
  20:  37.0
  21:  19.0
  22:  16.0
  23:  24.0
  24:  12.0
  25:  31.0
  26:  18.0
  27:  19.0
  28:  14.0
  29:  17.0
  30:  32.0
  31:  10.0
  32:  11.0
  33:  27.0
  34:  20.0
  35:  14.0
  36:  27.0
  37:  20.0
  38:  16.0
  39:  25.0
  40:  32.0
  41:  46.0
  42:  25.0
  43:  79.0
  44:  44.0
  45:  16.0
  46:  13.0
  47:  13.0
  48:  19.0
  49:  39.0
  50:  45.0
  51:  19.0
  52:  10.0
  53:  23.0
  54:  31.0
  55:  16.0
  56:  16.0
  57:  13.0
  58:  29.0
  59:  15.0
  60:  23.0
  61:  18.0
  62:  36.0
  63:  19.0
  64:  19.0
  65:  20.0
  66:  11.0
  67:  10.0
  68:  16.0
  69:  16.0
  70:  54.0
  71:  12.0
  72:  14.0
  73:  12.0
  74:  24.0
  75:  26.0


KeyboardInterrupt: 